In [1]:
import pandas as pd
import random
import networkx as nx
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

PAPERNUM = 100000
# 100,000

/Users/jyu7/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jyu7/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
# Load data
df = pd.read_csv('withRef.csv', nrows=PAPERNUM)
df.head(3)

/var/folders/vz/hrwwq3wn2ml3j1zbrk5s_x2m0000gp/T/ipykernel_70476/2038470307.py:2: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('withRef.csv', nrows=PAPERNUM)


,Unnamed: 0,id,title,year,n_citation,doc_type,reference_count,references,doi
0,0,1091,Preliminary Design of a Network Protocol Learn...,2013.0,1,Conference,2.0,2005687710;2018037215,https://doi.org/10.1007/978-3-642-39476-8_19
1,2,1674,A methodology for the physically accurate visu...,2011.0,1,Conference,15.0,1535888970;1992876689;1993710814;2035653341;20...,https://doi.org/10.2312/VAST/VAST11/137-144
2,3,1688,"Comparison of GARCH, Neural Network and Suppor...",2009.0,6,Conference,3.0,1560724230;1986968751;2156909104,https://doi.org/10.1007/978-3-642-11164-8_97


In [3]:
df.shape

(100000, 9)

## Graph

In [4]:
# Initialize graph and add nodes
G = nx.DiGraph()
G.add_nodes_from(df['id'].astype(str).tolist())
df['references'] = df['references'].astype(str).apply(lambda x: x.strip().split(';') if x else [])

In [5]:
# Add edges
for _, row in tqdm(df.iterrows(), total=len(df), desc="Building graph"):
    for ref in row['references']:
        if ref in G:
            G.add_edge(row['id'], ref)

Building graph: 100%|██████████| 100000/100000 [00:02<00:00, 47297.40it/s]


In [6]:
# Graph shape
print(len(G.nodes))
print(len(G.edges))

108077
33303


## Link Prediction

In [7]:
# Remove a fraction of edges for testing
def train_test_split_graph(G, test_ratio=0.3, seed=52):
    random.seed(seed)
    edges = list(G.edges())
    num_test = int(len(edges) * test_ratio)
    test_edges = random.sample(edges, num_test)
    train_graph = G.copy()
    train_graph.remove_edges_from(test_edges)
    return train_graph, test_edges

G_train, test_edges = train_test_split_graph(G)


In [8]:
# Pick random node pairs that are not connected to evaluate false positives
def generate_negative_edges(G, num_edges, excluded_edges):
    nodes = list(G.nodes())
    neg_edges = set()
    while len(neg_edges) < num_edges:
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v) and (u, v) not in excluded_edges:
            neg_edges.add((u, v))
    return list(neg_edges)

negative_edges = generate_negative_edges(G, len(test_edges), set(test_edges))

In [9]:
# Convert to undirected graphs
G_undirected = G.to_undirected()
G_train_undirected = G_train.to_undirected()

In [10]:
# Common Neighbors
pred_common = [
    (u, v, len(list(nx.common_neighbors(G_train_undirected, u, v))))
    for u, v in test_edges + negative_edges
    if u in G_train_undirected and v in G_train_undirected and len(list(nx.common_neighbors(G_train_undirected, u, v))) > 0
]
print(f"Number of Common Neighbors: {len(pred_common)}")

Number of Common Neighbors: 772


In [11]:
# Jaccard Coefficient
neighbors = {node: set(G_train_undirected.neighbors(node)) for node in G_train_undirected.nodes()}

def fast_jaccard(u, v):
    if u in neighbors and v in neighbors:
        inter = neighbors[u] & neighbors[v]
        union = neighbors[u] | neighbors[v]
        return (u, v, len(inter) / len(union)) if union else (u, v, 0.0)
    return (u, v, 0.0)

pred_jaccard = [fast_jaccard(u, v) for u, v in tqdm(test_edges + negative_edges, desc="Calculating Jaccard Coefficient", total=len(test_edges) + len(negative_edges))]

Calculating Jaccard Coefficient: 100%|██████████| 19980/19980 [00:00<00:00, 796265.76it/s]


In [12]:
# Adamic-Adar Index
pred_adamic = list(nx.adamic_adar_index(G_train_undirected, test_edges + negative_edges))

In [13]:
def evaluate_predictions(preds, true_edges_set):
    y_true = [(u, v) in true_edges_set for u, v, _ in preds]
    y_scores = [score for _, _, score in preds]
    return roc_auc_score(y_true, y_scores)

true_set = set(test_edges)
auc_jaccard = evaluate_predictions(pred_jaccard, true_set)
auc_adamic = evaluate_predictions(pred_adamic, true_set)

# AUC scores
print(f"AUC - Jaccard: {auc_jaccard:.4f}")
print(f"AUC - Adamic-Adar: {auc_adamic:.4f}")

AUC - Jaccard: 0.5386
AUC - Adamic-Adar: 0.5386


## Link Prediction With LLM

In [14]:
# SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight, fast

def get_title(node):
    if node not in df['id'].values:
        return ''
    title = df.loc[df['id'] == node, 'title'].values[0]
    return title

# Find unique nodes from edge pairs
unique_nodes = set(u for u, _ in test_edges + negative_edges).union(
               set(v for _, v in test_edges + negative_edges))

In [15]:
# Compute title embeddings for all nodes
node_embeddings = {node: model.encode(get_title(node), convert_to_tensor=True) for node in tqdm(unique_nodes, desc="Encoding titles", total=len(unique_nodes))}

Encoding titles: 100%|██████████| 31229/31229 [06:53<00:00, 75.47it/s]


In [16]:
# Compute similarity from embeddings
def cached_score(u, v):
    if u in node_embeddings and v in node_embeddings:
        return util.cos_sim(node_embeddings[u], node_embeddings[v]).item()
    return 0.5  # fallback for missing nodes

# Generate predictions
llm_preds = [(u, v, cached_score(u, v)) for u, v in tqdm(test_edges + negative_edges, desc="Calculating LLM scores", total=len(test_edges) + len(negative_edges))]

Calculating LLM scores: 100%|██████████| 19980/19980 [00:14<00:00, 1357.23it/s]


In [17]:
# AUC scores
auc_sbert = evaluate_predictions(llm_preds, set(test_edges))
print(f"AUC - SBERT: {auc_sbert:.4f}")

AUC - SBERT: 0.7160
